
# Fixation Time Analysis for Multilayer Evolutionary Games


---

## Overview

This notebook implements Monte Carlo simulations to estimate **fixation times** in evolutionary
game dynamics on **multilayer (multiplex) networks**. We study the death-birth Moran process
where strategies spread through a population based on fitness derived from game-theoretic payoffs.

### Key Concepts

- **Fixation Time (τ)**: The expected number of generations until the population reaches
  an absorbing state (all individuals adopt the same strategy)
- **Death-Birth (dB) Update**: A randomly selected individual dies and is replaced by an
  offspring from a neighbor, chosen proportionally to fitness
- **Multilayer Network**: Two coupled network layers where dynamics on each layer can
  influence the overall fitness of individuals

### Model Description

We consider a 2-layer multiplex network:

1. **Layer 1 (Game Layer)**: Individuals play a **Donation Game**
   - Cooperators (C) pay cost c=1 to provide benefit b to neighbors
   - Defectors (D) pay no cost and provide no benefit
   - Payoff: $u_i^{(1)} = -x_i + b \cdot \frac{\sum_{j \in N_i} w_{ij} x_j}{k_i}$

2. **Layer 2 (Constant Selection Layer)**: Individuals have type M or R
   - Type M (mutant): fitness contribution = r
   - Type R (resident): fitness contribution = 1
   - Payoff: $u_i^{(2)} = x_i(r-1) + 1$

3. **Combined Fitness**: $F_i = 1 + \delta(u_i^{(1)} + u_i^{(2)})$
   - δ: selection intensity (δ → 0 is weak selection)

### Absorbing States

The process terminates when both layers reach fixation:
- **AA**: All-C on Layer 1, All-M on Layer 2
- **AB**: All-C on Layer 1, All-R on Layer 2  
- **BA**: All-D on Layer 1, All-M on Layer 2
- **BB**: All-D on Layer 1, All-R on Layer 2

## Table of Contents

1. [Setup and Imports](#setup)
2. [Single-Layer Simulation](#single-layer)
3. [Multilayer Simulation](#multilayer)
4. [Parallelization Utilities](#parallel)
5. [Data Loading and Network Construction](#data-loading)
6. [Parameter Sweep Experiments](#experiments)
7. [Results Analysis](#results)

---

<a id='setup'></a>
## 1. Setup and Imports

Import required libraries and configure the computing environment.

In [ ]:
"""
Required Dependencies
---------------------
- numpy: Numerical computations and array operations
- networkx: Graph/network data structures and algorithms
- pandas: Data manipulation and CSV I/O
- multiprocessing: Parallel execution of simulations
"""

import time
import numpy as np
import networkx as nx
import pandas as pd
import multiprocessing as mp
from multiprocessing import get_context, cpu_count
from typing import Tuple, Dict, List, Optional
import warnings

# Configure multiprocessing for macOS compatibility
# The 'fork' method is required on macOS; 'spawn' is the default but causes issues
try:
    mp.set_start_method('fork')
except RuntimeError:
    # Method already set - this is expected in Jupyter environments
    pass

# Set random seed for reproducibility (comment out for true randomness)
# np.random.seed(42)

print(f"NumPy version: {np.__version__}")
print(f"NetworkX version: {nx.__version__}")
print(f"Available CPU cores: {cpu_count()}")

---

<a id='single-layer'></a>
## 2. Single-Layer Simulation

Implementation of the death-birth Moran process on a single network layer
with donation game payoffs. This serves as a baseline and building block
for the multilayer simulation.

In [ ]:
def simulate_single_layer_db(
    G: nx.Graph,
    b: float,
    delta: float,
    num_runs: int
) -> Tuple[float, float, float]:
    """
    Simulate the death-birth Moran process on a single-layer network.
    
    This function runs Monte Carlo simulations of evolutionary dynamics
    where individuals play a donation game and update strategies through
    a death-birth process.
    
    Parameters
    ----------
    G : networkx.Graph
        Undirected, unweighted graph representing the population structure.
        Must be connected; isolated nodes will cause issues.
    b : float
        Benefit parameter in the donation game (cost c is fixed to 1).
        Cooperation is favored when b > c (i.e., b > 1).
    delta : float
        Selection intensity. Controls the strength of selection:
        - delta → 0: weak selection (nearly neutral drift)
        - delta → ∞: strong selection (deterministic dynamics)
    num_runs : int
        Number of independent simulation runs for Monte Carlo averaging.
    
    Returns
    -------
    avg_time : float
        Unconditional mean fixation time (averaged over all absorbing states).
    avg_time_allC : float
        Mean fixation time conditional on all-Cooperator fixation.
        Returns NaN if no runs ended in all-C.
    avg_time_allD : float
        Mean fixation time conditional on all-Defector fixation.
        Returns NaN if no runs ended in all-D.
    
    Notes
    -----
    The simulation initializes with a single cooperator placed uniformly
    at random in a population of defectors.
    
    Payoff Calculation:
        u_i = -x_i + b * (sum of cooperating neighbors) / degree_i
        where x_i = 1 for cooperators, 0 for defectors
    
    Fitness Calculation:
        F_i = 1 + delta * u_i
    
    Update Rule (Death-Birth):
        1. Select a random node d to "die"
        2. Select a parent from neighbors of d with probability ∝ fitness
        3. d adopts the parent's strategy
    
    Examples
    --------
    >>> G = nx.complete_graph(10)
    >>> avg, avg_C, avg_D = simulate_single_layer_db(G, b=2.0, delta=0.01, num_runs=1000)
    >>> print(f"Mean fixation time: {avg:.1f}")
    """
    # =========================================================================
    # PREPROCESSING: Build efficient data structures for simulation
    # =========================================================================
    
    node_list = list(G.nodes())
    n = len(node_list)
    node_to_idx = {node: i for i, node in enumerate(node_list)}
    
    # Precompute neighbor indices for O(1) lookup during simulation
    # This avoids repeated dictionary lookups in the inner loop
    neighbor_indices = [
        [node_to_idx[v] for v in G.neighbors(u)]
        for u in node_list
    ]
    
    # Precompute degrees (number of neighbors per node)
    deg = np.array([len(neighs) for neighs in neighbor_indices], dtype=float)
    
    # =========================================================================
    # SIMULATION: Run Monte Carlo trials
    # =========================================================================
    
    times = []           # Fixation times for all runs
    times_to_allC = []   # Fixation times conditioned on all-C outcome
    times_to_allD = []   # Fixation times conditioned on all-D outcome
    
    for run in range(num_runs):
        # ----- Step 1: Initialize population -----
        # Strategy array: 0 = Defector, 1 = Cooperator
        x = np.zeros(n, dtype=int)
        # Place one cooperator at a uniformly random position
        x[np.random.randint(n)] = 1
        
        t = 0  # Time step counter
        
        while True:
            # ----- Step 2: Check for absorption -----
            num_cooperators = x.sum()
            
            if num_cooperators == 0 or num_cooperators == n:
                # Population has fixated
                times.append(t)
                if num_cooperators == n:
                    times_to_allC.append(t)
                else:
                    times_to_allD.append(t)
                break
            
            # ----- Step 3: Calculate payoffs -----
            payoff = np.zeros(n, dtype=float)
            
            for k in range(n):
                if deg[k] == 0:
                    # Isolated node: no interactions, zero payoff
                    payoff[k] = 0.0
                else:
                    # Count cooperating neighbors (weighted by edge if applicable)
                    coop_neighbors = sum(x[j] for j in neighbor_indices[k])
                    # Donation game payoff:
                    # Cooperator pays cost 1, all neighbors receive benefit b/degree
                    payoff[k] = -x[k] + b * (coop_neighbors / deg[k])
            
            # ----- Step 4: Calculate fitness (fecundity) -----
            F = 1.0 + delta * payoff
            
            # ----- Step 5: Death-birth update -----
            # Death: select a random node to be replaced
            d = np.random.randint(n)
            nbrs = neighbor_indices[d]
            
            if not nbrs:
                # Isolated node - skip this time step
                # (should not happen in connected graphs)
                t += 1
                continue
            
            # Birth: select parent from neighbors proportional to fitness
            F_neighbors = F[nbrs]
            probs = F_neighbors / F_neighbors.sum()
            parent = np.random.choice(nbrs, p=probs)
            
            # Offspring inherits parent's strategy
            x[d] = x[parent]
            
            t += 1
    
    # =========================================================================
    # RESULTS: Compute summary statistics
    # =========================================================================
    
    avg_time = float(np.mean(times))
    avg_C = float(np.mean(times_to_allC)) if times_to_allC else float('nan')
    avg_D = float(np.mean(times_to_allD)) if times_to_allD else float('nan')
    
    return avg_time, avg_C, avg_D

---

<a id='multilayer'></a>
## 3. Multilayer Simulation

Extension to two coupled network layers where:
- Layer 1: Donation game dynamics (C vs D)
- Layer 2: Constant selection dynamics (M vs R)

Both layers share the same set of nodes but may have different edge structures.
The combined fitness influences reproduction on both layers.

In [ ]:
def simulate_multilayer_db(
    G1: nx.Graph,
    G2: nx.Graph,
    b: float,
    r: float,
    delta: float,
    num_runs: int
) -> Tuple[
    float,                    # avg_time_all
    Dict[str, float],         # avg_time_cond (joint states)
    float,                    # avg_game_fix
    float,                    # avg_const_fix
    Dict[str, float],         # game_fix_cond
    Dict[str, float]          # const_fix_cond
]:
    """
    Simulate death-birth Moran process on a 2-layer multiplex network.
    
    This function models evolutionary dynamics where individuals exist on two
    coupled network layers. Each layer has its own topology and game, but
    fitness is computed from contributions of both layers.
    
    Parameters
    ----------
    G1 : networkx.Graph
        Layer 1 network (game layer). Nodes must match G2.
        Edges can have 'weight' attribute; defaults to 1.0.
    G2 : networkx.Graph
        Layer 2 network (constant selection layer). Nodes must match G1.
        Edges can have 'weight' attribute; defaults to 1.0.
    b : float
        Benefit parameter for the donation game on Layer 1.
    r : float
        Relative fitness of mutant type M on Layer 2.
        - r > 1: M is advantageous
        - r = 1: neutral selection
        - r < 1: M is disadvantageous
    delta : float
        Selection intensity. Controls strength of selection vs drift.
    num_runs : int
        Number of independent Monte Carlo simulation runs.
    
    Returns
    -------
    avg_time_all : float
        Unconditional mean time to joint absorption (both layers fixated).
    avg_time_cond : dict
        Mean joint fixation times conditioned on each absorbing state.
        Keys: 'AA' (C+M), 'AB' (C+R), 'BA' (D+M), 'BB' (D+R).
    avg_game_fix : float
        Unconditional mean time for Layer 1 to fixate.
    avg_const_fix : float
        Unconditional mean time for Layer 2 to fixate.
    game_fix_cond : dict
        Layer 1 fixation times conditioned on outcome.
        Keys: 'C' (all cooperators), 'D' (all defectors).
    const_fix_cond : dict
        Layer 2 fixation times conditioned on outcome.
        Keys: 'M' (all mutants), 'R' (all residents).
    
    Notes
    -----
    Layer Dynamics:
        - Layer 1 (Game): Individuals play donation game.
          Payoff u1_i = -x1_i + b * (weighted fraction of C neighbors)
        
        - Layer 2 (Constant Selection): Type determines fitness contribution.
          Payoff u2_i = x2_i * (r - 1) + 1
          This gives u2 = r for type M (x2=1) and u2 = 1 for type R (x2=0).
    
    Combined Fitness:
        F_i = 1 + delta * (u1_i + u2_i)
    
    Update Mechanism:
        Each time step, one death-birth event occurs on EACH layer:
        1. Layer 1: Random death, fitness-weighted birth from L1 neighbors
        2. Layer 2: Random death, fitness-weighted birth from L2 neighbors
    
    Examples
    --------
    >>> G1 = nx.complete_graph(10)
    >>> G2 = nx.cycle_graph(10)
    >>> results = simulate_multilayer_db(G1, G2, b=2.0, r=1.1, delta=0.01, num_runs=500)
    >>> avg_all, cond, game_t, const_t, game_cond, const_cond = results
    """
    # =========================================================================
    # VALIDATION AND PREPROCESSING
    # =========================================================================
    
    N = G1.number_of_nodes()
    assert G2.number_of_nodes() == N, "Both layers must have same number of nodes"
    
    nodes = list(G1.nodes())
    idx = {u: i for i, u in enumerate(nodes)}
    
    def prepare_layer(G: nx.Graph) -> Tuple[List, np.ndarray]:
        """
        Precompute neighbor lists and weighted degrees for efficient simulation.
        
        Returns
        -------
        neigh : list of (neighbor_indices, weights) tuples
        deg : ndarray of weighted degrees
        """
        neigh = []
        deg = np.zeros(N)
        
        for u in nodes:
            nbrs = []    # Neighbor indices
            ws = []      # Edge weights
            
            for v, data in G[u].items():
                nbrs.append(idx[v])
                ws.append(data.get('weight', 1.0))
            
            neigh.append((nbrs, np.array(ws)))
            deg[idx[u]] = sum(ws)  # Weighted degree
        
        return neigh, deg
    
    neigh1, deg1 = prepare_layer(G1)
    neigh2, deg2 = prepare_layer(G2)
    
    # =========================================================================
    # STORAGE CONTAINERS
    # =========================================================================
    
    # Joint fixation statistics
    total_times = []                                       # All joint fixation times
    times_cond = {lbl: [] for lbl in ('AA', 'AB', 'BA', 'BB')}  # By absorbing state
    
    # Per-layer fixation statistics
    game_fix_times = []    # Time for Layer 1 to fixate
    game_fix_state = []    # Final state of Layer 1: 'C' or 'D'
    const_fix_times = []   # Time for Layer 2 to fixate
    const_fix_state = []   # Final state of Layer 2: 'M' or 'R'
    
    # =========================================================================
    # MONTE CARLO SIMULATION
    # =========================================================================
    
    for _ in range(num_runs):
        # ----- Initialize strategies -----
        # Layer 1: 0 = Defector (D), 1 = Cooperator (C)
        x1 = np.zeros(N, dtype=int)
        x1[np.random.randint(N)] = 1  # One random cooperator
        
        # Layer 2: 0 = Resident (R), 1 = Mutant (M)
        x2 = np.zeros(N, dtype=int)
        x2[np.random.randint(N)] = 1  # One random mutant
        
        t = 0              # Time step counter
        t_game = None      # Time when Layer 1 fixates
        t_const = None     # Time when Layer 2 fixates
        state1 = None      # Final state of Layer 1
        state2 = None      # Final state of Layer 2
        
        while True:
            # ----- Check Layer 1 fixation -----
            sum1 = x1.sum()
            if t_game is None and (sum1 == 0 or sum1 == N):
                t_game = t
                state1 = 'C' if sum1 == N else 'D'
            
            # ----- Check Layer 2 fixation -----
            sum2 = x2.sum()
            if t_const is None and (sum2 == 0 or sum2 == N):
                t_const = t
                state2 = 'M' if sum2 == N else 'R'
            
            # ----- Check joint absorption -----
            all_C = (sum1 == N)
            all_D = (sum1 == 0)
            all_M = (sum2 == N)
            all_R = (sum2 == 0)
            
            if (all_C or all_D) and (all_M or all_R):
                # Determine joint absorbing state label
                if all_C and all_M:
                    lbl = 'AA'
                elif all_C and all_R:
                    lbl = 'AB'
                elif all_D and all_M:
                    lbl = 'BA'
                else:
                    lbl = 'BB'
                
                # Record joint statistics
                total_times.append(t)
                times_cond[lbl].append(t)
                
                # Record per-layer statistics
                game_fix_times.append(t_game if t_game is not None else t)
                game_fix_state.append(state1 or ('C' if all_C else 'D'))
                const_fix_times.append(t_const if t_const is not None else t)
                const_fix_state.append(state2 or ('M' if all_M else 'R'))
                break
            
            # ----- Calculate Layer 1 payoffs (Donation Game) -----
            u1 = np.zeros(N)
            for i in range(N):
                nbrs, ws = neigh1[i]
                if deg1[i] > 0:
                    # Weighted fraction of cooperating neighbors
                    frac_coop = ws.dot(x1[nbrs]) / deg1[i]
                    # Payoff: -cost if cooperator + benefit from cooperating neighbors
                    u1[i] = -x1[i] + b * frac_coop
            
            # ----- Calculate Layer 2 payoffs (Constant Selection) -----
            # u2 = r for mutants (x2=1), u2 = 1 for residents (x2=0)
            u2 = x2 * (r - 1) + 1
            
            # ----- Calculate combined fitness -----
            F = 1 + delta * (u1 + u2)
            
            # ----- Death-birth update on Layer 1 -----
            d1 = np.random.randint(N)
            nbrs1, _ = neigh1[d1]
            if nbrs1:
                probs1 = F[nbrs1] / F[nbrs1].sum()
                parent1 = np.random.choice(nbrs1, p=probs1)
                x1[d1] = x1[parent1]
            
            # ----- Death-birth update on Layer 2 -----
            d2 = np.random.randint(N)
            nbrs2, _ = neigh2[d2]
            if nbrs2:
                probs2 = F[nbrs2] / F[nbrs2].sum()
                parent2 = np.random.choice(nbrs2, p=probs2)
                x2[d2] = x2[parent2]
            
            t += 1
    
    # =========================================================================
    # AGGREGATE RESULTS
    # =========================================================================
    
    # Joint fixation statistics
    avg_time_all = float(np.mean(total_times))
    avg_time_cond = {
        lbl: (float(np.mean(times_cond[lbl])) if times_cond[lbl] else float('nan'))
        for lbl in times_cond
    }
    
    # Per-layer unconditional statistics
    avg_game_fix = float(np.mean(game_fix_times))
    avg_const_fix = float(np.mean(const_fix_times))
    
    # Per-layer conditional statistics
    game_fix_cond = {
        'C': float(np.mean([t for t, s in zip(game_fix_times, game_fix_state) if s == 'C'])
                   if any(s == 'C' for s in game_fix_state) else np.nan),
        'D': float(np.mean([t for t, s in zip(game_fix_times, game_fix_state) if s == 'D'])
                   if any(s == 'D' for s in game_fix_state) else np.nan)
    }
    
    const_fix_cond = {
        'M': float(np.mean([t for t, s in zip(const_fix_times, const_fix_state) if s == 'M'])
                   if any(s == 'M' for s in const_fix_state) else np.nan),
        'R': float(np.mean([t for t, s in zip(const_fix_times, const_fix_state) if s == 'R'])
                   if any(s == 'R' for s in const_fix_state) else np.nan)
    }
    
    return (
        avg_time_all,
        avg_time_cond,
        avg_game_fix,
        avg_const_fix,
        game_fix_cond,
        const_fix_cond
    )

---

<a id='parallel'></a>
## 4. Parallelization Utilities

Functions to distribute simulation workload across multiple CPU cores.
This significantly speeds up Monte Carlo experiments with many runs.

In [ ]:
def _run_multilayer_chunk(args: tuple) -> tuple:
    """
    Worker function for parallel execution.
    
    Unpacks arguments and calls the main simulation function.
    This indirection is required for multiprocessing.Pool.map().
    
    Parameters
    ----------
    args : tuple
        (G1, G2, b, r, delta, num_runs) - arguments for simulate_multilayer_db
    
    Returns
    -------
    tuple
        Results from simulate_multilayer_db
    """
    return simulate_multilayer_db(*args)


# Register worker function in __main__ namespace for spawn-based multiprocessing
# This is required on some platforms (Windows, macOS with spawn)
import __main__
__main__._run_multilayer_chunk = _run_multilayer_chunk


def parallel_multilayer_simulation(
    G1: nx.Graph,
    G2: nx.Graph,
    b: float,
    r: float,
    delta: float,
    total_runs: int,
    n_jobs: Optional[int] = None
) -> Tuple[
    float,                    # avg_time_all
    Dict[str, float],         # avg_time_cond
    float,                    # avg_game_fix
    float,                    # avg_const_fix
    Dict[str, float],         # game_fix_cond
    Dict[str, float]          # const_fix_cond
]:
    """
    Parallel wrapper for multilayer simulation using multiprocessing.
    
    Distributes simulation runs across multiple CPU cores and aggregates
    results using weighted averaging.
    
    Parameters
    ----------
    G1, G2 : networkx.Graph
        Layer 1 and Layer 2 networks.
    b : float
        Donation game benefit parameter.
    r : float
        Constant selection relative fitness.
    delta : float
        Selection intensity.
    total_runs : int
        Total number of simulation runs (distributed across workers).
    n_jobs : int, optional
        Number of parallel workers. Defaults to number of CPU cores.
    
    Returns
    -------
    Same as simulate_multilayer_db - aggregated across all parallel runs.
    
    Notes
    -----
    The function uses the 'fork' context for multiprocessing, which is
    required on macOS and generally more efficient for NumPy-heavy workloads.
    
    Runs are distributed as evenly as possible across workers. If total_runs
    is not evenly divisible by n_jobs, some workers get one extra run.
    
    Examples
    --------
    >>> G1 = nx.complete_graph(20)
    >>> G2 = nx.complete_graph(20)
    >>> results = parallel_multilayer_simulation(
    ...     G1, G2, b=2.0, r=1.1, delta=0.01,
    ...     total_runs=10000, n_jobs=4
    ... )
    """
    if n_jobs is None:
        n_jobs = cpu_count()
    
    # Distribute runs across workers
    base, remainder = divmod(total_runs, n_jobs)
    runs_per_worker = [base + (1 if i < remainder else 0) for i in range(n_jobs)]
    
    # Create task arguments for each worker
    tasks = [
        (G1, G2, b, r, delta, runs_per_worker[i])
        for i in range(n_jobs)
    ]
    
    # Execute in parallel using fork context
    ctx = get_context('fork')
    with ctx.Pool(processes=n_jobs) as pool:
        results = pool.map(_run_multilayer_chunk, tasks)
    
    # =========================================================================
    # AGGREGATE RESULTS (weighted average by number of runs per worker)
    # =========================================================================
    
    # Joint fixation time (unconditional)
    avg_time_all = sum(
        results[i][0] * runs_per_worker[i] for i in range(n_jobs)
    ) / total_runs
    
    # Joint fixation time (conditional on absorbing state)
    avg_time_cond = {}
    for lbl in ('AA', 'AB', 'BA', 'BB'):
        weighted_sum = sum(
            results[i][1][lbl] * runs_per_worker[i] for i in range(n_jobs)
        )
        avg_time_cond[lbl] = weighted_sum / total_runs
    
    # Per-layer fixation times (unconditional)
    avg_game_fix = sum(
        results[i][2] * runs_per_worker[i] for i in range(n_jobs)
    ) / total_runs
    
    avg_const_fix = sum(
        results[i][3] * runs_per_worker[i] for i in range(n_jobs)
    ) / total_runs
    
    # Per-layer fixation times (conditional)
    game_fix_cond = {}
    for lbl in ('C', 'D'):
        weighted_sum = sum(
            results[i][4][lbl] * runs_per_worker[i] for i in range(n_jobs)
        )
        game_fix_cond[lbl] = weighted_sum / total_runs
    
    const_fix_cond = {}
    for lbl in ('M', 'R'):
        weighted_sum = sum(
            results[i][5][lbl] * runs_per_worker[i] for i in range(n_jobs)
        )
        const_fix_cond[lbl] = weighted_sum / total_runs
    
    return (
        avg_time_all,
        avg_time_cond,
        avg_game_fix,
        avg_const_fix,
        game_fix_cond,
        const_fix_cond
    )

---

<a id='data-loading'></a>
## 5. Data Loading and Network Construction

Utilities for loading empirical network data from CSV files
and constructing NetworkX graph objects for simulation.

In [ ]:
def load_multilayer_network(
    filepath: str,
    layer1_id: int = 1,
    layer2_id: int = 2
) -> Tuple[nx.Graph, nx.Graph]:
    """
    Load a multilayer network from a CSV edge list file.
    
    Expected CSV format:
        i,j,w,layer
        1,2,1.0,1
        1,3,0.5,1
        ...
    
    Where:
        - i, j: node IDs (integers)
        - w: edge weight (float)
        - layer: layer identifier (integer)
    
    Parameters
    ----------
    filepath : str
        Path to the CSV file containing edge list.
    layer1_id : int, default=1
        Layer identifier for Layer 1 edges in the CSV.
    layer2_id : int, default=2
        Layer identifier for Layer 2 edges in the CSV.
    
    Returns
    -------
    G1 : networkx.Graph
        Layer 1 network with weighted edges.
    G2 : networkx.Graph
        Layer 2 network with weighted edges.
        Both graphs contain the union of nodes from both layers.
    
    Examples
    --------
    >>> G1, G2 = load_multilayer_network("network_edges.csv")
    >>> print(f"Layer 1: {G1.number_of_nodes()} nodes, {G1.number_of_edges()} edges")
    """
    # Read edge list CSV
    edge_df = pd.read_csv(
        filepath,
        comment="#",                           # Skip comment lines
        names=["i", "j", "w", "layer"],        # Column names
        sep=",",
        skip_blank_lines=True,
        dtype={"i": int, "j": int, "w": float, "layer": int}
    )
    
    # Filter to only the requested layers
    edge_df = edge_df[edge_df["layer"].isin([layer1_id, layer2_id])]
    
    def build_layer_graph(df: pd.DataFrame, layer_id: int) -> nx.Graph:
        """
        Construct a NetworkX graph from edges belonging to a specific layer.
        """
        layer_edges = df[df["layer"] == layer_id]
        G = nx.Graph()
        G.add_weighted_edges_from(
            layer_edges[["i", "j", "w"]].itertuples(index=False, name=None)
        )
        return G
    
    G1 = build_layer_graph(edge_df, layer1_id)
    G2 = build_layer_graph(edge_df, layer2_id)
    
    # Ensure both layers have the same node set
    # (important for multiplex: same individuals on both layers)
    all_nodes = set(G1.nodes()) | set(G2.nodes())
    G1.add_nodes_from(all_nodes)
    G2.add_nodes_from(all_nodes)
    
    return G1, G2


def print_network_summary(G1: nx.Graph, G2: nx.Graph, name: str = "Network") -> None:
    """
    Print summary statistics for a multilayer network.
    
    Parameters
    ----------
    G1, G2 : networkx.Graph
        The two network layers.
    name : str
        Name to display in the summary.
    """
    print(f"\n{'='*50}")
    print(f"{name} Summary")
    print(f"{'='*50}")
    print(f"Number of nodes: {G1.number_of_nodes()}")
    print(f"\nLayer 1:")
    print(f"  Edges: {G1.number_of_edges()}")
    print(f"  Avg degree: {2*G1.number_of_edges()/G1.number_of_nodes():.2f}")
    print(f"  Connected: {nx.is_connected(G1)}")
    print(f"\nLayer 2:")
    print(f"  Edges: {G2.number_of_edges()}")
    print(f"  Avg degree: {2*G2.number_of_edges()/G2.number_of_nodes():.2f}")
    print(f"  Connected: {nx.is_connected(G2)}")
    print(f"{'='*50}\n")

---

<a id='experiments'></a>
## 6. Parameter Sweep Experiments

Functions to run systematic experiments across parameter grids
and save results for later analysis.

In [ ]:
def run_parameter_sweep(
    G1: nx.Graph,
    G2: nx.Graph,
    b_values: List[float],
    r_values: List[float],
    delta_values: List[float],
    total_runs: int,
    n_jobs: Optional[int] = None,
    network_name: str = "network",
    output_file: Optional[str] = None,
    verbose: bool = True
) -> pd.DataFrame:
    """
    Run a systematic parameter sweep over (b, r, delta) combinations.
    
    For each parameter combination, runs parallel simulations and records
    all fixation time statistics.
    
    Parameters
    ----------
    G1, G2 : networkx.Graph
        Layer 1 and Layer 2 networks.
    b_values : list of float
        Benefit parameter values to sweep.
    r_values : list of float
        Relative fitness parameter values to sweep.
    delta_values : list of float
        Selection intensity values to sweep.
    total_runs : int
        Number of Monte Carlo runs per parameter combination.
    n_jobs : int, optional
        Number of parallel workers (default: CPU count).
    network_name : str, default="network"
        Name identifier for the network (included in results).
    output_file : str, optional
        If provided, save results to this CSV file.
    verbose : bool, default=True
        Print progress updates during sweep.
    
    Returns
    -------
    df : pandas.DataFrame
        Results dataframe with columns:
        - network: network identifier
        - b, r, delta: parameter values
        - avg_time_all: unconditional joint fixation time
        - time_AA, time_AB, time_BA, time_BB: conditional joint times
        - avg_game_fix, avg_const_fix: per-layer unconditional times
        - game_fix_C, game_fix_D: Layer 1 conditional times
        - const_fix_M, const_fix_R: Layer 2 conditional times
        - wall_sec: wall-clock time for this parameter combination
    
    Examples
    --------
    >>> results = run_parameter_sweep(
    ...     G1, G2,
    ...     b_values=[1.0, 2.0, 3.0],
    ...     r_values=[1.0, 1.1],
    ...     delta_values=[0.01, 0.02],
    ...     total_runs=5000,
    ...     output_file="results.csv"
    ... )
    """
    if n_jobs is None:
        n_jobs = cpu_count()
    
    records = []
    total_combos = len(b_values) * len(r_values) * len(delta_values)
    combo_idx = 0
    
    for b in b_values:
        for r in r_values:
            for delta in delta_values:
                combo_idx += 1
                
                # Run parallel simulation
                t_start = time.perf_counter()
                (
                    avg_all,
                    avg_cond,
                    avg_game,
                    avg_const,
                    game_cond,
                    const_cond
                ) = parallel_multilayer_simulation(
                    G1, G2, b, r, delta, total_runs, n_jobs
                )
                wall_time = time.perf_counter() - t_start
                
                # Record results
                record = {
                    "network": network_name,
                    "b": b,
                    "r": r,
                    "delta": delta,
                    "avg_time_all": avg_all,
                    "time_AA": avg_cond["AA"],
                    "time_AB": avg_cond["AB"],
                    "time_BA": avg_cond["BA"],
                    "time_BB": avg_cond["BB"],
                    "avg_game_fix": avg_game,
                    "avg_const_fix": avg_const,
                    "game_fix_C": game_cond["C"],
                    "game_fix_D": game_cond["D"],
                    "const_fix_M": const_cond["M"],
                    "const_fix_R": const_cond["R"],
                    "wall_sec": wall_time,
                }
                records.append(record)
                
                if verbose:
                    print(
                        f"[{combo_idx}/{total_combos}] "
                        f"{network_name}  b={b:<4} r={r:<4} δ={delta:<5}  "
                        f"⟨τ⟩={avg_all:.1f}  ({wall_time:.1f}s)"
                    )
    
    # Create DataFrame
    df = pd.DataFrame(records)
    
    # Optionally save to file
    if output_file:
        df.to_csv(output_file, index=False)
        if verbose:
            print(f"\nResults saved to: {output_file}")
    
    return df

---

## Quick Demo: Validation on Complete Graphs

Run a small test to verify the implementation works correctly.

In [ ]:
# Create test networks (complete graphs)
N_TEST = 10
G1_test = nx.complete_graph(N_TEST)
G2_test = nx.complete_graph(N_TEST)

# Test parameters
b_test = 2.0      # Benefit (b > 1 favors cooperation)
r_test = 1.1      # Mutant fitness advantage
delta_test = 0.02 # Weak selection
runs_test = 1000  # Number of simulation runs

print("Running validation test...")
print(f"Network: Complete graph with {N_TEST} nodes")
print(f"Parameters: b={b_test}, r={r_test}, δ={delta_test}")
print(f"Runs: {runs_test}\n")

# Run simulation
t_start = time.perf_counter()
(
    avg_all,
    avg_cond,
    avg_game,
    avg_const,
    game_cond,
    const_cond
) = parallel_multilayer_simulation(
    G1_test, G2_test, b_test, r_test, delta_test, runs_test, n_jobs=4
)
elapsed = time.perf_counter() - t_start

# Display results
print("=" * 50)
print("RESULTS")
print("=" * 50)
print(f"\nJoint Fixation:")
print(f"  Unconditional ⟨τ⟩: {avg_all:.2f}")
print(f"  Conditional:")
for lbl, val in avg_cond.items():
    print(f"    {lbl}: {val:.2f}")

print(f"\nLayer 1 (Game) Fixation:")
print(f"  Unconditional ⟨τ⟩: {avg_game:.2f}")
print(f"  Conditional on C: {game_cond['C']:.2f}")
print(f"  Conditional on D: {game_cond['D']:.2f}")

print(f"\nLayer 2 (Constant) Fixation:")
print(f"  Unconditional ⟨τ⟩: {avg_const:.2f}")
print(f"  Conditional on M: {const_cond['M']:.2f}")
print(f"  Conditional on R: {const_cond['R']:.2f}")

print(f"\nWall time: {elapsed:.2f} seconds")
print("=" * 50)

---

<a id='results'></a>
## 7. Results Analysis (Template)

Template code for loading and analyzing saved results.
Uncomment and modify as needed for your specific analysis.

In [ ]:
# ============================================================================
# TEMPLATE: Load and analyze saved results
# ============================================================================

# Uncomment to load results from a CSV file:
# df_results = pd.read_csv("multilayer_sim_results.csv")
# print(df_results.head())

# Example: Filter results for specific parameters
# df_filtered = df_results[df_results['delta'] == 0.01]

# Example: Pivot table for heatmap visualization
# pivot = df_results.pivot_table(
#     values='avg_time_all',
#     index='b',
#     columns='r',
#     aggfunc='mean'
# )

print("Results analysis section - customize as needed.")

---

## Example: Running on Empirical Network Data

Template for loading empirical networks and running parameter sweeps.
Modify the file path and parameters for your specific use case.

In [ ]:
# ============================================================================
# TEMPLATE: Run on empirical network
# ============================================================================

# Uncomment and modify for your network file:

# # Load empirical network
# G_emp1, G_emp2 = load_multilayer_network(
#     "network.csv/law_edges.csv",
#     layer1_id=1,
#     layer2_id=2
# )
# print_network_summary(G_emp1, G_emp2, "Law Network")

# # Define parameter grid
# B_VALUES = [1.0, 2.0, 3.0]
# R_VALUES = [1.0, 1.1, 1.2, 3.0]
# DELTA_VALUES = [0.01, 0.02, 0.2]
# TOTAL_RUNS = 5000

# # Run parameter sweep
# df_empirical = run_parameter_sweep(
#     G_emp1, G_emp2,
#     b_values=B_VALUES,
#     r_values=R_VALUES,
#     delta_values=DELTA_VALUES,
#     total_runs=TOTAL_RUNS,
#     network_name="law_empirical",
#     output_file="law_empirical_results.csv"
# )

print("Empirical network template - uncomment and configure as needed.")

---

## References

1. Moran, P. A. P. (1958). Random processes in genetics. *Mathematical Proceedings of the Cambridge Philosophical Society*, 54(1), 60-71.

2. Nowak, M. A., Tarnita, C. E., & Antal, T. (2010). Evolutionary dynamics in structured populations. *Philosophical Transactions of the Royal Society B*, 365(1537), 19-30.

3. Allen, B., et al. (2017). Evolutionary dynamics on any population structure. *Nature*, 544(7649), 227-230.

---

## Appendix: Implementation Notes

### Performance Considerations

- **Precomputation**: Neighbor lists and degrees are precomputed before simulation loops to minimize overhead.
- **Parallelization**: Uses Python's multiprocessing with 'fork' context for efficient shared memory.
- **Memory**: Each worker maintains independent state; total memory scales linearly with n_jobs.

### Known Limitations

- Graphs must be connected; isolated nodes may cause infinite loops.
- The 'fork' context may not work on Windows (use 'spawn' with appropriate modifications).
- Very large networks (N > 10,000) may require optimized implementations.

